# Autonomous Driving Gym: Reward Shaping & Domain Randomization

## Description


This notebook provides a comprehensive guide to **Reward Shaping** and **Domain Randomization** within the Autonomous Driving Gym environment. It aims to support the development of an RL policy that enables an autonomous vehicle to maintain a recommended speed (cruise control) and follow the lane center (lane keeping).

The notebook mainly covers the following key activities:

1.  **Reward Shaping**:
    *   Demonstrates the behavior of internal lane keeping and cruise control policies.
    *   Explains and implements reward shaping specifically for lane keeping, where the agent controls steering while cruise control is handled internally.
    *   Explains and implements reward shaping for cruise control, where the agent controls acceleration/braking while lane keeping is handled internally.
    *   Introduces a combined reward wrapper that incorporates both lane keeping and cruise control objectives, allowing the agent to control both steering and acceleration/braking.
2.  **Domain Randomization**:
    *   Introduces the concept of domain randomization to improve the generalization of the trained RL policy.
    *   Provides a `DomainRandomizationWrapper` to generate random road geometries during training.
    *   Includes examples of different randomization parameters to create varying road complexities.
    *   Sets up separate environments for training (with randomization) and evaluation (with a fixed, potentially difficult road).

**RL Policy Training and Evaluation**:

-   Demonstrates how to train a PPO agent using the shaped rewards and the domain-randomized environment.
-   Provides helper functions to evaluate the trained policy on both training and evaluation environments.
-   Includes visualization tools to plot trajectories and time series results.
-   Introduces performance metrics to quantitatively assess the policy's performance based on distance to the lane center and speed.

This is a long notebook, so please utilize the table of contents on the left for navigation (≡). The "Editing guidance" section highlights the parts of the notebook that are most critical for you to modify and experiment with.

## **Editing guidance**

The frequently asked question that this "editing guidance" is answering is:

**"Which parts of the notebook should I change? Because sometimes it feels like there are 10-20 critical lines of code to changes out of >1000 lines of code!"**

We fully appreciate and agreed with the sentiment of this FAQ! Hence, the following is the list of sections within this notebook that are critical for you to edit so that you gain the intended experiece with PID synthesis and evaluation.

**Note:** working through the whole notebook is still relevant for your understanding (even though you might not need to edit the parts not listed below)!

**Note:** The (link to section) hyperlink may not work outside of google colab.

---

Here are the key sections where you can experiment and make changes:

*   **Reward Shaping (Part 1)**:
    *   **Lane Keeping Reward Shaping**: Decide which observations to use [here](#scrollTo=Observation_Parameters). Experiment with the `LaneKeepRewardWrapper` to shape the reward for lane keeping. You can modify the `k_distance` parameter and add other reward terms based on observations relevant to staying in the lane center. ([link to section](#scrollTo=Custom_Reward_Wrapper_Focusing_on_Lane_Keeping))
    *   **Cruise Control Reward Shaping**: Decide which observations to use again. Modify the `CruiseControlRewardWrapper` to shape the reward for cruise control. You can adjust the `target_speed_mps`, `use_recommended`, `k_speed`, and introduce penalties for deviations from the target speed or recommended speed. ([link to section](#scrollTo=Reward_Wrapper_Focusing_on_Cruise_Control))
    *   **Combined Reward Wrapper**: Experiment with the `CombinedRewardWrapper` by adjusting the weights (`k_distance`, `k_speed`) of the lane keeping and cruise control rewards, and by adding new reward components that consider the main as well as secondary objectives. ([link to section](#scrollTo=Combined_Reward_Wrapper))

*   **Domain Randomization (Part 2)**:
    *   **Road Randomization Parameters**: Modify the `road_randomization_params` dictionary to control the generation of random road geometries. Experiment with different ranges for the number of elements, straight lengths, curvatures, and angles to create various training environments. You can also experiment further with custom rewards here since domain randomization usually makes the models more robust. ([link to section](#scrollTo=Domain_Randomization_Wrapper_))
    *   **Model Type**: You can experiment with different RL model types from `stable_baselines3` (e.g., `SAC`, `DDPG`, etc.) instead of PPO for training the policy. Alternatively, you can also update / configure the underlying neural network architectures of these models. Note that on-policy algorithms such as PPO are generally faster to train. ([link to section](#scrollTo=Model_Definition_and_Training_))

*   **Performance Metrics**:
    *   Modify the `compute_performance_metrics_from_time_series` function to define and calculate custom performance metrics that are relevant to your goals (e.g., smoothness of steering, efficiency of acceleration/braking, time spent off-road). ([link to section](#scrollTo=Define_a_performance_Metrics_Function_per_simulation_time_series_))

# Dependencies (install and clone)

In [ ]:
!pip install -q gymnasium
!pip install -q stable_baselines3

In [ ]:
%cd /content
%rm -rf ai4r-gym
!git clone https://gitlab.unimelb.edu.au/ai4r/ai4r-gym.git

In [ ]:
%cd /content/ai4r-gym
%pwd
!git status

## Library Imports

In [ ]:
import ai4rgym
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
from stable_baselines3 import PPO, SAC, DDPG
from utils import ensure_dir, ensure_dirs, eval_model, evaluate_policy
from utils import plot_rewards, plot_and_animate_trajectory

## Environment Settings

### Specify the Vehicle Parameters

Dictionary with car specifications,
in the form of a dynamic bicycle model

In [ ]:
# SPECIFY THE VEHCILE PARAMETERS
bicycle_model_parameters = {
    "Lf" : 0.55*2.875,
    "Lr" : 0.45*2.875,
    "m"  : 2000.0,
    "Iz" : (1.0/12.0) * 2000.0 * (4.692**2+1.850**2),
    "Cm" : (1.0/100.0) * (1.0 * 400.0 * 9.0) / 0.2286,
    "Cd" : 0.5 * 0.24 * 2.2204 * 1.202,
    "delta_offset" : 0 * np.pi/180,
    "delta_request_max" : 45 * np.pi/180,
    "Ddelta_lower_limit" : -45 * np.pi/180,
    "Ddelta_upper_limit" :  45 * np.pi/180,
    "v_transition_min" : 500.0 / 3.6,
    "v_transition_max" : 600.0 / 3.6,
    "body_len_f" : (0.55*2.875) * 1.5,
    "body_len_r" : (0.45*2.875) * 1.5,
    "body_width" : 2.50,
}

### Specify the Road


Specified as a list of dictionaries, where each
element in the list specifies a segment of the road.
Example segment dictionaries:
```Python
{"type":"straight", "length":3.0}
{"type":"curved", "curvature":1/50.0, "angle_in_degrees":45.0}
{"type":"curved", "curvature":1/50.0, "length":30.0}
```

In [ ]:
road_elements_list = [
    {"type":"straight", "length":100.0, "v_max_kph":60.0},
    {"type":"curved", "curvature":1/800.0, "angle_in_degrees":15.0, "v_max_kph":50.0},
    {"type":"straight", "length":100.0, "v_max_kph":60.0},
    {"type":"curved", "curvature":-1/400.0, "angle_in_degrees":30.0, "v_max_kph":40.0},
    {"type":"straight", "length":100.0, "v_max_kph":60.0},
]

### Specify the Numerical Integration Details

The options available for the numerical
integration method are:
`["euler", "huen", "midpoint", "rk4", "rk45"]`


In [ ]:
numerical_integration_parameters = {
    "method" : "rk4",
    "Ts" : 0.05,
    "num_steps_per_Ts" : 1,
}

###  Specify the Initial State Distribution

The initial state is sampled from a uniform
distribution between the minimum and maximum
(i.e., between lower and upper bounds)
> Note: a factor of (1/3.6) converts from units of [km/h] to [m/s]

In [ ]:
py_init_min = -1.0
py_init_max =  1.0

v_init_min_in_kmh = 55.0
v_init_max_in_kmh = 65.0

py_init_min = -1.0
py_init_max =  1.0

v_init_min_in_kmh = 55.0
v_init_max_in_kmh = 65.0

initial_state_bounds = {
    "px_init_min" : 0.0,
    "px_init_max" : 0.0,
    "py_init_min" : py_init_min,
    "py_init_max" : py_init_max,
    "theta_init_min" : 0.0,
    "theta_init_max" : 0.0,
    "vx_init_min" : v_init_min_in_kmh * (1.0/3.6),
    "vx_init_max" : v_init_max_in_kmh * (1.0/3.6),
    "vy_init_min" : 0.0,
    "vy_init_max" : 0.0,
    "omega_init_min" : 0.0,
    "omega_init_max" : 0.0,
    "delta_init_min" : 0.0,
    "delta_init_max" : 0.0,
}

### Specify the Observation Parameters

In [ ]:
# SPECIFY THE OBSERVATION PARAMETERS
observation_parameters = {
    "should_include_ground_truth_px"                       :  "info",
    "should_include_ground_truth_py"                       :  "info",
    "should_include_ground_truth_theta"                    :  "info",
    "should_include_ground_truth_vx"                       :  "info",
    "should_include_ground_truth_vy"                       :  "info",
    "should_include_ground_truth_omega"                    :  "info",
    "should_include_ground_truth_delta"                    :  "info",
    "should_include_road_progress_at_closest_point"        :  "info",
    "should_include_vx_sensor"                             :  "obs",
    "should_include_distance_to_closest_point"             :  "obs",
    "should_include_heading_angle_relative_to_line"        :  "obs",
    "should_include_heading_angular_rate_gyro"             :  "info",
    "should_include_closest_point_coords_in_body_frame"    :  "info",
    "should_include_look_ahead_line_coords_in_body_frame"  :  "info",
    "should_include_road_curvature_at_closest_point"       :  "obs",
    "should_include_look_ahead_road_curvatures"            :  "info",
    "should_include_recommended_speed"                     :  "obs",

    "scaling_for_ground_truth_px"                       :  1.0,
    "scaling_for_ground_truth_py"                       :  1.0,
    "scaling_for_ground_truth_theta"                    :  1.0,
    "scaling_for_ground_truth_vx"                       :  1.0,
    "scaling_for_ground_truth_vy"                       :  1.0,
    "scaling_for_ground_truth_omega"                    :  1.0,
    "scaling_for_ground_truth_delta"                    :  1.0,
    "scaling_for_road_progress_at_closest_point"        :  1.0,
    "scaling_for_vx_sensor"                             :  1.0,
    "scaling_for_distance_to_closest_point"             :  1.0,
    "scaling_for_heading_angle_relative_to_line"        :  1.0,
    "scaling_for_heading_angular_rate_gyro"             :  1.0,
    "scaling_for_closest_point_coords_in_body_frame"    :  1.0,
    "scaling_for_look_ahead_line_coords_in_body_frame"  :  1.0,
    "scaling_for_road_curvature_at_closest_point"       :  1.0,
    "scaling_for_look_ahead_road_curvatures"            :  1.0,

    "vx_sensor_bias"    : 0.0,
    "vx_sensor_stddev"  : 0.1,

    "distance_to_closest_point_bias"    :  0.0,
    "distance_to_closest_point_stddev"  :  0.01,

    "heading_angle_relative_to_line_bias"    :  0.0,
    "heading_angle_relative_to_line_stddev"  :  0.01,

    "heading_angular_rate_gyro_bias"    :  0.0,
    "heading_angular_rate_gyro_stddev"  :  0.01,

    "closest_point_coords_in_body_frame_bias"    :  0.0,
    "closest_point_coords_in_body_frame_stddev"  :  0.0,

    "look_ahead_line_coords_in_body_frame_bias"    :  0.0,
    "look_ahead_line_coords_in_body_frame_stddev"  :  0.0,

    "road_curvature_at_closest_point_bias"    :  0.0,
    "road_curvature_at_closest_point_stddev"  :  0.0,

    "look_ahead_road_curvatures_bias"    :  0.0,
    "look_ahead_road_curvatures_stddev"  :  0.0,

    "look_ahead_line_coords_in_body_frame_distance"    :  100.0,
    "look_ahead_line_coords_in_body_frame_num_points"  :  10,
}

### Specify Termination Parameters

In [ ]:
termination_parameters = {
    "speed_lower_bound"  :  0.0,
    "speed_upper_bound"  :  (200.0/3.6),
    "distance_to_closest_point_upper_bound"  :  20.0,
    "reward_speed_lower_bound"  :  0.0,
    "reward_speed_upper_bound"  :  0.0,
    "reward_distance_to_closest_point_upper_bound"  :  0.0,
}

## Part 1: Reward Shaping


### 1) Demo: Internal Lane Keeping and Cruise Control

We enable both internal controllers and set `action_interface="auto"`, which collapses the external action to a no‑op. A neutral policy will send zeros; the internals will drive.


In [ ]:
# Build environment (reuse dictionaries defined above)
from evaluation.evaluation_for_autonomous_driving import simulate_policy, plot_results_from_time_series_dict, plot_road_from_list_of_road_elements

# Helper policy that returns a neutral action
class NoOpPolicy:
    def compute_action(self, observation, info_dict, terminated, truncated):
        # Action shape is determined by the env's action_interface
        # For "none": shape is (1,) and value is ignored
        import numpy as np
        low = env.action_space.low
        high = env.action_space.high
        return ((low + high) * 0.0).astype('float32')

# Internal policies: both enabled
internal_policy_config_both = {
    "enable_lane_keep": True,
    "enable_cruise_control": True,
    "action_interface": "none",  # derives to "none" when both are enabled
}

env = gym.make(
    "ai4rgym/autonomous_driving_env",
    render_mode=None,
    bicycle_model_parameters=bicycle_model_parameters,
    road_elements_list=road_elements_list,
    numerical_integration_parameters=numerical_integration_parameters,
    termination_parameters=termination_parameters,
    initial_state_bounds=initial_state_bounds,
    observation_parameters=observation_parameters,
    internal_policy_config=internal_policy_config_both,
)

# Optional surface condition
env.unwrapped.set_road_condition(road_condition="dry")

# Run simulation with neutral action policy (let internals drive)
N_sim = 1500
results = simulate_policy(env, N_sim, NoOpPolicy(), seed=42, should_save_look_ahead_results=False, should_save_observations=True, verbose=1)

save_dir = "models/internal_policies"
ensure_dir(save_dir)

# Quick trajectory/time-series plots
plot_results_from_time_series_dict(env, results, save_dir, file_name_suffix="internal_policies", should_plot_reward=False)



### 2) Lane keeping reward shaping (cruise control internal)

We enable internal cruise control so the longitudinal speed is handled by the environment. We define a `LaneKeepRewardWrapper` that rewards staying near the lane center. A simple steering policy is provided as a working baseline; students are encouraged to iterate on the reward.



#### Observation Parameters

We restrict observations to the signals relevant for lane keeping. You can edit this dictionary.
- `distance_to_closest_point` (obs)
- `heading_angle_relative_to_line` (obs)

We explicitly disable unrelated default observations (e.g., `vx_sensor`, `recommended_speed`) to keep the input minimal.


In [ ]:

# Minimal observation dictionary for lane keeping
observation_parameters_lk = {
    "should_include_ground_truth_px"                       :  "info",
    "should_include_ground_truth_py"                       :  "info",
    "should_include_ground_truth_theta"                    :  "info",
    "should_include_ground_truth_vx"                       :  "info",
    "should_include_ground_truth_vy"                       :  "info",
    "should_include_ground_truth_omega"                    :  "info",
    "should_include_ground_truth_delta"                    :  "info",
    "should_include_road_progress_at_closest_point"        :  "info",
    "should_include_vx_sensor"                             :  "info",
    "should_include_distance_to_closest_point"             :  "obs",
    "should_include_heading_angle_relative_to_line"        :  "obs",
    "should_include_heading_angular_rate_gyro"             :  "info",
    "should_include_closest_point_coords_in_body_frame"    :  "info",
    "should_include_look_ahead_line_coords_in_body_frame"  :  "info",
    "should_include_road_curvature_at_closest_point"       :  "info",
    "should_include_look_ahead_road_curvatures"            :  "info",

    "scaling_for_ground_truth_px"                       :  1.0,
    "scaling_for_ground_truth_py"                       :  1.0,
    "scaling_for_ground_truth_theta"                    :  1.0,
    "scaling_for_ground_truth_vx"                       :  1.0,
    "scaling_for_ground_truth_vy"                       :  1.0,
    "scaling_for_ground_truth_omega"                    :  1.0,
    "scaling_for_ground_truth_delta"                    :  1.0,
    "scaling_for_road_progress_at_closest_point"        :  1.0,
    "scaling_for_vx_sensor"                             :  1.0,
    "scaling_for_distance_to_closest_point"             :  1.0,
    "scaling_for_heading_angle_relative_to_line"        :  1.0,
    "scaling_for_heading_angular_rate_gyro"             :  1.0,
    "scaling_for_closest_point_coords_in_body_frame"    :  1.0,
    "scaling_for_look_ahead_line_coords_in_body_frame"  :  1.0,
    "scaling_for_road_curvature_at_closest_point"       :  1.0,
    "scaling_for_look_ahead_road_curvatures"            :  1.0,

    "vx_sensor_bias"    : 0.0,
    "vx_sensor_stddev"  : 0.1,

    "distance_to_closest_point_bias"    :  0.0,
    "distance_to_closest_point_stddev"  :  0.01,

    "heading_angle_relative_to_line_bias"    :  0.0,
    "heading_angle_relative_to_line_stddev"  :  0.01,

    "heading_angular_rate_gyro_bias"    :  0.0,
    "heading_angular_rate_gyro_stddev"  :  0.01,

    "closest_point_coords_in_body_frame_bias"    :  0.0,
    "closest_point_coords_in_body_frame_stddev"  :  0.0,

    "look_ahead_line_coords_in_body_frame_bias"    :  0.0,
    "look_ahead_line_coords_in_body_frame_stddev"  :  0.0,

    "road_curvature_at_closest_point_bias"    :  0.0,
    "road_curvature_at_closest_point_stddev"  :  0.0,

    "look_ahead_road_curvatures_bias"    :  0.0,
    "look_ahead_road_curvatures_stddev"  :  0.0,

    "look_ahead_line_coords_in_body_frame_distance"    :  100.0,
    "look_ahead_line_coords_in_body_frame_num_points"  :  10,
}

💡
- **Think**: *What observations are most relevant for lane keeping?*
- **Note**: It's always good to start simple and add additional observations as needed

####  Custom Reward Wrapper Focusing on Lane Keeping

In [ ]:

import gymnasium as gym
import numpy as np

class LaneKeepRewardWrapper(gym.Wrapper):
    """Reward based on distance to lane center (smaller is better)."""
    def __init__(self, env, k_distance: float = 1.0):
        super().__init__(env)
        self.k_distance = float(k_distance)
    def reset(self, **kwargs):
        return super().reset(**kwargs)
    def step(self, action):
        obs, base_r, terminated, truncated, info = self.env.step(action)
        # Strict access: these keys must exist in the env's ground-truth dict
        gt = self.env.unwrapped.current_ground_truth
        d = float(abs(gt["distance_to_closest_point"]))
        # Reward: larger when closer to center; tapers off with distance
        r = self.k_distance * (1.0 / (1.0 + d))
        return obs, r, terminated, truncated, info


#### 💡 Possible options to consider:

- [ ] *pick 1-2 changes from below and try to implement*

---

- Very high reward for being near the lane center
- Smaller reward for staying on the lane/road
- Penalty for large steering actions
- Quadratic penalty / rewards for stronger learning/reward signals
- Penalty for going off-road
- Penalty for large heading angle relative to lane direction


> (including but not limited to, feel free to innovate!)

#### Environment Setup for Lane Keeping (Cruise Control Internal)

In [ ]:

# Internal policy: cruise control on, lane-keeping off (agent controls steering)
internal_policy_config_lk = {
    "enable_lane_keep": False,
    "enable_cruise_control": True,
    "action_interface": "steer_only",
    # Optionally tune cruise control target speed
    "cruise_use_recommended_speed": True,
    "cruise_target_speed_mps": 60.0 / 3.6,
}

# Build env and wrap with lane-keeping reward
import gymnasium as gym
from evaluation.evaluation_for_autonomous_driving import simulate_policy, plot_results_from_time_series_dict

env_lk = gym.make(
    "ai4rgym/autonomous_driving_env",
    render_mode=None,
    bicycle_model_parameters=bicycle_model_parameters,
    road_elements_list=road_elements_list,
    numerical_integration_parameters=numerical_integration_parameters,
    termination_parameters=termination_parameters,
    initial_state_bounds=initial_state_bounds,
    observation_parameters=observation_parameters_lk,
    internal_policy_config=internal_policy_config_lk,
)

env_lk = LaneKeepRewardWrapper(env_lk, k_distance=1.0)

#### Training RL Policy for Lane Keeping

In [ ]:

# Train a small PPO agent to verify the shaped reward works
from stable_baselines3 import PPO
import gymnasium as gym

# Rescale the 1-D steering action to [-1, 1] for PPO stability
env_lk_train = gym.wrappers.RescaleAction(env_lk, min_action=-1, max_action=1)

# Rename model/run for this experiment
logdir = "logs"
model_name_lk = "PPO_LK"
models_dir_lk = f"models/{model_name_lk}"
figs_dir_lk = f"{models_dir_lk}/figs"
ensure_dirs([models_dir_lk, figs_dir_lk])

TIMESTEPS_PER_EPOCH = 50000

# Build model and train using the same epoch/timestep structure
model_lk = PPO("MultiInputPolicy", env_lk_train, verbose=1, tensorboard_log=logdir)

start_epoch = 1
N_EPOCHS_LK = 10  # follow previous pattern; adjust if desired

for i in range(start_epoch, N_EPOCHS_LK + 1):
    model_lk.learn(
        total_timesteps=TIMESTEPS_PER_EPOCH,
        reset_num_timesteps=False,
        tb_log_name=f"{model_name_lk}",
    )
    step_count = TIMESTEPS_PER_EPOCH * i
    model_lk.save(f"{models_dir_lk}/{step_count}")
    eval_model(env_lk_train, model_lk, figs_dir_lk, step_count, plot_velocity=True)
    print(f"[LK] Finished epoch {i}/{N_EPOCHS_LK} | saved at step {step_count}")

💡 Look inside the `models/{model_name}/figs` directory for visualizations and insights related to the model's performance, such as - reward values, speed, trajectory, etc.


### 3) Cruise control reward shaping (lane keeping internal)

We enable internal lane keeping so the lateral control is handled by the environment. We define a `CruiseControlRewardWrapper` that rewards staying near a target speed (or the recommended speed). A simple throttle policy is provided as a baseline.



#### Observation Parameters

We restrict observations to the signals relevant for cruise control shaping:
- `vx_sensor` (obs): longitudinal speed measurement used by the agent/policy.
- `recommended_speed` (info): ground-truth recommended speed from the road geometry.

We keep other recommendations in `info` (not observations) to keep the
learning target simple; the reward wrapper can still access ground-truth values.

In [ ]:

# Minimal observation dictionary for lane keeping
observation_parameters_cc = {
    "should_include_ground_truth_px"                       :  "info",
    "should_include_ground_truth_py"                       :  "info",
    "should_include_ground_truth_theta"                    :  "info",
    "should_include_ground_truth_vx"                       :  "info",
    "should_include_ground_truth_vy"                       :  "info",
    "should_include_ground_truth_omega"                    :  "info",
    "should_include_ground_truth_delta"                    :  "info",
    "should_include_road_progress_at_closest_point"        :  "info",
    "should_include_vx_sensor"                             :  "obs",
    "should_include_distance_to_closest_point"             :  "info",
    "should_include_heading_angle_relative_to_line"        :  "info",
    "should_include_heading_angular_rate_gyro"             :  "info",
    "should_include_closest_point_coords_in_body_frame"    :  "info",
    "should_include_look_ahead_line_coords_in_body_frame"  :  "info",
    "should_include_road_curvature_at_closest_point"       :  "info",
    "should_include_look_ahead_road_curvatures"            :  "info",
    "should_include_recommended_speed"                     :  "obs",
    "should_include_speed_limit"                           :  "info",

    "scaling_for_ground_truth_px"                       :  1.0,
    "scaling_for_ground_truth_py"                       :  1.0,
    "scaling_for_ground_truth_theta"                    :  1.0,
    "scaling_for_ground_truth_vx"                       :  1.0,
    "scaling_for_ground_truth_vy"                       :  1.0,
    "scaling_for_ground_truth_omega"                    :  1.0,
    "scaling_for_ground_truth_delta"                    :  1.0,
    "scaling_for_road_progress_at_closest_point"        :  1.0,
    "scaling_for_vx_sensor"                             :  1.0,
    "scaling_for_distance_to_closest_point"             :  1.0,
    "scaling_for_heading_angle_relative_to_line"        :  1.0,
    "scaling_for_heading_angular_rate_gyro"             :  1.0,
    "scaling_for_closest_point_coords_in_body_frame"    :  1.0,
    "scaling_for_look_ahead_line_coords_in_body_frame"  :  1.0,
    "scaling_for_road_curvature_at_closest_point"       :  1.0,
    "scaling_for_look_ahead_road_curvatures"            :  1.0,

    "vx_sensor_bias"    : 0.0,
    "vx_sensor_stddev"  : 0.1,

    "distance_to_closest_point_bias"    :  0.0,
    "distance_to_closest_point_stddev"  :  0.01,

    "heading_angle_relative_to_line_bias"    :  0.0,
    "heading_angle_relative_to_line_stddev"  :  0.01,

    "heading_angular_rate_gyro_bias"    :  0.0,
    "heading_angular_rate_gyro_stddev"  :  0.01,

    "closest_point_coords_in_body_frame_bias"    :  0.0,
    "closest_point_coords_in_body_frame_stddev"  :  0.0,

    "look_ahead_line_coords_in_body_frame_bias"    :  0.0,
    "look_ahead_line_coords_in_body_frame_stddev"  :  0.0,

    "road_curvature_at_closest_point_bias"    :  0.0,
    "road_curvature_at_closest_point_stddev"  :  0.0,

    "look_ahead_road_curvatures_bias"    :  0.0,
    "look_ahead_road_curvatures_stddev"  :  0.0,

    "look_ahead_line_coords_in_body_frame_distance"    :  100.0,
    "look_ahead_line_coords_in_body_frame_num_points"  :  10,
}

💡
- **Think**: *What observations are most relevant for cruise control?*
- **Note**: It's always good to start simple and add additional observations as needed

#### Reward Wrapper Focusing on Cruise Control

In [ ]:

import gymnasium as gym
import numpy as np

class CruiseControlRewardWrapper(gym.Wrapper):
    """Reward based on proximity to a target speed (m/s), gated by road heading."""
    def __init__(self, env, target_speed_mps: float = 60.0/3.6, use_recommended: bool = False, k_speed: float = 1.0):
        super().__init__(env)
        self.target_speed_mps = float(target_speed_mps)
        self.use_recommended = bool(use_recommended)
        self.k_speed = float(k_speed)

    def reset(self, **kwargs):
        return super().reset(**kwargs)

    def step(self, action):
        obs, base_r, terminated, truncated, info = self.env.step(action)

        # Ground-truth values from env
        v = float(self.env.unwrapped.car.vx)
        gt = self.env.unwrapped.current_ground_truth
        v_rec = float(gt['recommended_speed_at_closest_point'])
        v_tgt = v_rec if (self.use_recommended and v_rec > 0.0) else self.target_speed_mps

        # Heading relative to road line (assumed radians)
        theta = float(gt['heading_angle_relative_to_line'])
        gate = max(0.0, np.cos(theta))  # 1 when aligned, 0 when ≥90° off

        r_speed = self.k_speed * (1.0 / (1.0 + abs(v - v_tgt)))
        r = gate * r_speed

        # Debug info (optional, drop if too noisy)
        info = dict(info, heading_rel_line=theta, heading_gate=gate, r_speed=r_speed)

        return obs, r, terminated, truncated, info

#### 💡 Possible options to consider:

- [ ] *pick 1-2 changes from below and try to implement*

---

- High reward for being near the recommended speed
- Flat reward closer to the recommended speed to encourage practical driving
- Smaller reward for going above the recommended speed
- Large penalty for exceeding the speed limit
- Penalty for going below a certain speed
- Penalty for large acceleration actions
- Quadratic penalty / rewards for stronger learning/reward signals

> (including but not limited to, feel free to innovate!)

#### Environment Setup for Cruise Control (Lane Keeping Internal)

In [ ]:

# Internal policy: lane keeping on, cruise control off (agent controls drive)
internal_policy_config_cc = {
    "enable_lane_keep": True,
    "enable_cruise_control": False,
    "action_interface": "drive_only",
}

import gymnasium as gym
from evaluation.evaluation_for_autonomous_driving import simulate_policy, plot_results_from_time_series_dict

env_cc = gym.make(
    "ai4rgym/autonomous_driving_env",
    render_mode=None,
    bicycle_model_parameters=bicycle_model_parameters,
    road_elements_list=road_elements_list,
    numerical_integration_parameters=numerical_integration_parameters,
    termination_parameters=termination_parameters,
    initial_state_bounds=initial_state_bounds,
    observation_parameters=observation_parameters_cc,
    internal_policy_config=internal_policy_config_cc,
)

env_cc = CruiseControlRewardWrapper(env_cc, target_speed_mps=60.0/3.6, use_recommended=True, k_speed=1.0)


#### Training RL Policy for Cruise Control

In [ ]:
# Train a small PPO agent to verify the shaped reward works
from stable_baselines3 import PPO
import gymnasium as gym

# Rescale the 1-D drive action to [-1, 1] for PPO stability
env_cc_train = gym.wrappers.RescaleAction(env_cc, min_action=-1, max_action=1)

logdir = "logs"

# Rename model/run for this experiment
model_name_cc = "PPO_CC"
models_dir_cc = f"models/{model_name_cc}"
figs_dir_cc = f"{models_dir_cc}/figs"
ensure_dirs([models_dir_cc, figs_dir_cc])

TIMESTEPS_PER_EPOCH = 50000

# Build model and train using the same epoch/timestep structure
model_cc = PPO("MultiInputPolicy", env_cc_train, verbose=1, tensorboard_log=logdir)

start_epoch = 1
N_EPOCHS_CC = 10  # follow previous pattern; adjust if desired

for i in range(start_epoch, N_EPOCHS_CC + 1):
    model_cc.learn(
        total_timesteps=TIMESTEPS_PER_EPOCH,
        reset_num_timesteps=False,
        tb_log_name=f"{model_name_cc}",
    )
    step_count = TIMESTEPS_PER_EPOCH * i
    model_cc.save(f"{models_dir_cc}/{step_count}")
    eval_model(env_cc_train, model_cc, figs_dir_cc, step_count, plot_velocity=True)
    print(f"[CC] Finished epoch {i}/{N_EPOCHS_CC} | saved at step {step_count}")


### 4) Combine lane-keeping and cruise rewards

We combine the rewards into a single `CombinedRewardWrapper`.  
Additional observations can be enabled to support both tasks.

In [ ]:

observation_parameters = {
    "should_include_ground_truth_px"                       :  "info",
    "should_include_ground_truth_py"                       :  "info",
    "should_include_ground_truth_theta"                    :  "info",
    "should_include_ground_truth_vx"                       :  "info",
    "should_include_ground_truth_vy"                       :  "info",
    "should_include_ground_truth_omega"                    :  "info",
    "should_include_ground_truth_delta"                    :  "info",
    "should_include_road_progress_at_closest_point"        :  "info",
    "should_include_vx_sensor"                             :  "obs",
    "should_include_distance_to_closest_point"             :  "obs",
    "should_include_heading_angle_relative_to_line"        :  "obs",
    "should_include_heading_angular_rate_gyro"             :  "info",
    "should_include_closest_point_coords_in_body_frame"    :  "info",
    "should_include_look_ahead_line_coords_in_body_frame"  :  "info",
    "should_include_road_curvature_at_closest_point"       :  "info",
    "should_include_look_ahead_road_curvatures"            :  "info",
    "should_include_recommended_speed"                     :  "obs",
    "should_include_speed_limit"                           :  "info",

    "scaling_for_ground_truth_px"                       :  1.0,
    "scaling_for_ground_truth_py"                       :  1.0,
    "scaling_for_ground_truth_theta"                    :  1.0,
    "scaling_for_ground_truth_vx"                       :  1.0,
    "scaling_for_ground_truth_vy"                       :  1.0,
    "scaling_for_ground_truth_omega"                    :  1.0,
    "scaling_for_ground_truth_delta"                    :  1.0,
    "scaling_for_road_progress_at_closest_point"        :  1.0,
    "scaling_for_vx_sensor"                             :  1.0,
    "scaling_for_distance_to_closest_point"             :  1.0,
    "scaling_for_heading_angle_relative_to_line"        :  1.0,
    "scaling_for_heading_angular_rate_gyro"             :  1.0,
    "scaling_for_closest_point_coords_in_body_frame"    :  1.0,
    "scaling_for_look_ahead_line_coords_in_body_frame"  :  1.0,
    "scaling_for_road_curvature_at_closest_point"       :  1.0,
    "scaling_for_look_ahead_road_curvatures"            :  1.0,

    "vx_sensor_bias"    : 0.0,
    "vx_sensor_stddev"  : 0.1,

    "distance_to_closest_point_bias"    :  0.0,
    "distance_to_closest_point_stddev"  :  0.01,

    "heading_angle_relative_to_line_bias"    :  0.0,
    "heading_angle_relative_to_line_stddev"  :  0.01,

    "heading_angular_rate_gyro_bias"    :  0.0,
    "heading_angular_rate_gyro_stddev"  :  0.01,

    "closest_point_coords_in_body_frame_bias"    :  0.0,
    "closest_point_coords_in_body_frame_stddev"  :  0.0,

    "look_ahead_line_coords_in_body_frame_bias"    :  0.0,
    "look_ahead_line_coords_in_body_frame_stddev"  :  0.0,

    "road_curvature_at_closest_point_bias"    :  0.0,
    "road_curvature_at_closest_point_stddev"  :  0.0,

    "look_ahead_road_curvatures_bias"    :  0.0,
    "look_ahead_road_curvatures_stddev"  :  0.0,

    "look_ahead_line_coords_in_body_frame_distance"    :  100.0,
    "look_ahead_line_coords_in_body_frame_num_points"  :  10,
}

#### Combined Reward Wrapper

**Note:** When combining multiple reward values, it's important that they are balanced (i.e. one reward term does not dominate / have very low values). You can use the k_terms to balance multiple rewards. Both distance and speed rewards are kept between 0-1 in this example and appear in similar scales. It's a good idea to keep reward values within gentle bounds (while also having sufficient resolution) to promote stability.   

In [ ]:
import gymnasium as gym
import numpy as np

class CombinedRewardWrapper(gym.Wrapper):
    """
    Reward combining lane keeping (distance to center) and cruise control
    (target speed with heading gating).
    """
    def __init__(self, env,
                 k_distance: float = 1.0,
                 k_speed: float = 1.0,
                 target_speed_mps: float = 60.0/3.6,
                 use_recommended: bool = False):
        super().__init__(env)
        self.k_distance = float(k_distance)
        self.k_speed = float(k_speed)
        self.target_speed_mps = float(target_speed_mps)
        self.use_recommended = bool(use_recommended)

    def reset(self, **kwargs):
        return super().reset(**kwargs)

    def step(self, action):
        obs, base_r, terminated, truncated, info = self.env.step(action)

        # Ground-truth values
        gt = self.env.unwrapped.current_ground_truth
        v = float(self.env.unwrapped.car.vx)
        d = float(abs(gt["distance_to_closest_point"]))
        v_rec = float(gt["recommended_speed_at_closest_point"])
        theta = float(gt["heading_angle_relative_to_line"])

        # Lane-keeping reward: closer to center → larger reward
        r_lane = self.k_distance * (1.0 / (1.0 + d))

        # Cruise-control reward: closer to target speed (with heading gating)
        v_tgt = v_rec if (self.use_recommended and v_rec > 0.0) else self.target_speed_mps
        gate = max(0.0, np.cos(theta))  # downweight when poorly aligned
        r_speed = self.k_speed * (1.0 / (1.0 + abs(v - v_tgt)))

        # Final reward: sum of both
        r = r_lane + gate * r_speed

        # Add debug info
        info = dict(info,
                    lane_distance=d,
                    r_lane=r_lane,
                    heading_rel_line=theta,
                    heading_gate=gate,
                    r_speed=r_speed,
                    v=v,
                    v_tgt=v_tgt)

        return obs, r, terminated, truncated, info


- [ ] Combine the updates you've already made to the Lane Keep / Cruise Control Rewards here
- [ ] Optionally, you can explore additional reward terms (see below)

#### 💡 Further Improvements to reward:

> (including but not limited to, feel free to innovate!)

- Bonus for smooth driving (e.g., low jerk/acceleration changes)
- Penalty for high steering angles at high speeds
- Penalty for rapid deceleration/breaking
- Bonus reward for progress along the lane
- Termination penalty for going off-road
- Bonus reward for finishing the track/road

> See week 4 notebook (`ad_gym_vanilla_rl_v2025_08_19.ipynb`) for an example of some additional reward terms, such as termination penalties, progress rewards, quadratic rewards/penalties, etc.

**Important Note** While adding additional reward terms can help guide the learning process, it's crucial to strike a balance. Overcomplicating the reward structure may lead to unintended behaviors or make it harder for the agent to learn effectively. Start with simple reward components that work well and iteratively refine them based on observed performance and learning outcomes.

**Watch Out for Reward Exploitation!**:
Certain rewards may be 'gamed' by the agent, leading to unintended behaviors. Some examples -
- if a reward is given for progress along the lane, the agent might learn to exploit this by making rapid, erratic movements that maximize this reward without actually driving safely.
- if a penalty is given for large steering angles, the agent might learn to drive very slowly to avoid making any significant turns, which could be impractical in real-world scenarios.
- if a high reward is given for driving at a specified speed, the agent might learn to accelerate rapidly to that speed and then maintain it, potentially ignoring other important aspects of driving.
- if a higher weight is given for speed reward, the agent might try to drive fast but in the wrong direction to maximize reward. or the agent might even drive in circles at high speed to maximize reward.

Always monitor the agent's behavior and adjust the reward structure as necessary to ensure it aligns with desired outcomes.
If you observe any cases of reward exploitation, that would also be a great learning experience and that deserves a mention in your report!

#### Initialise the Autonomous Driving Environment

In [ ]:
# No internal controllers; agent controls both steering and throttle
internal_policy_config_combined = {
    "enable_lane_keep": False,
    "enable_cruise_control": False,
    "action_interface": "full",
}

env_comb = gym.make(
    "ai4rgym/autonomous_driving_env",
    render_mode=None,
    bicycle_model_parameters=bicycle_model_parameters,
    road_elements_list=road_elements_list,
    numerical_integration_parameters=numerical_integration_parameters,
    termination_parameters=termination_parameters,
    initial_state_bounds=initial_state_bounds,
    observation_parameters=observation_parameters,
    internal_policy_config=internal_policy_config_combined,
)

env_comb = CombinedRewardWrapper(env_comb, k_distance=1.0, k_speed=1.0, target_speed_mps=60.0/3.6, use_recommended=True)

#### Training RL Policy for Combined Lane Keeping and Cruise Control

In [ ]:
# Train a small PPO agent to verify the shaped reward works
from stable_baselines3 import PPO
import gymnasium as gym

# Rescale the 1-D drive action to [-1, 1] for PPO stability
env_comb_train = gym.wrappers.RescaleAction(
    env_comb, min_action=-1, max_action=1)

logdir = "logs"

# Rename model/run for this experiment
model_name = "PPO_combined"
models_dir = f"models/{model_name}"
figs_dir = f"{models_dir}/figs"
ensure_dirs([models_dir, figs_dir])

TIMESTEPS_PER_EPOCH = 50000

# Build model and train using the same epoch/timestep structure
model = PPO("MultiInputPolicy", env_comb_train, verbose=1, tensorboard_log=logdir)

start_epoch = 1
N_EPOCHS_CC = 10  # follow previous pattern; adjust if desired

for i in range(start_epoch, N_EPOCHS_CC + 1):
    model.learn(
        total_timesteps=TIMESTEPS_PER_EPOCH,
        reset_num_timesteps=False,
        tb_log_name=f"{model_name}",
    )
    step_count = TIMESTEPS_PER_EPOCH * i
    model.save(f"{models_dir}/{step_count}")
    eval_model(env_comb_train, model, figs_dir, step_count, plot_velocity=True)
    print(f"[CC] Finished epoch {i}/{N_EPOCHS_CC} | saved at step {step_count}")

## Part 2 - Domain Randomization

In [ ]:
internal_policy_config = {
    "enable_lane_keep": False,
    "enable_cruise_control": False,
    "action_interface": "full",
}

Helper function for environment creation

In [ ]:
def create_env(road_elements_list):
    env = gym.make(
        "ai4rgym/autonomous_driving_env",
        render_mode=None,
        bicycle_model_parameters=bicycle_model_parameters,
        road_elements_list=road_elements_list,
        numerical_integration_parameters=numerical_integration_parameters,
        termination_parameters=termination_parameters,
        initial_state_bounds=initial_state_bounds,
        observation_parameters=observation_parameters,
        internal_policy_config=internal_policy_config,
    )

    # Set the integration method and time step
    Ts_sim = 0.05
    integration_method = "rk4"
    env.unwrapped.set_integration_method(integration_method)
    env.unwrapped.set_integration_Ts(Ts_sim)

    # Set the road condition
    env.unwrapped.set_road_condition(road_condition="wet")

    # Rescale actions and wrap the environment
    env = gym.wrappers.RescaleAction(env, min_action=-1, max_action=1)
    env = CombinedRewardWrapper(env)
    return env

### Domain Randomization Wrapper:

In [ ]:
from ai4rgym.envs.road import Road
import random

class DomainRandomizationWrapper(gym.Wrapper):
    def __init__(self, env, road_randomization_params=None):
        super(DomainRandomizationWrapper, self).__init__(env)
        self.road_randomization_params = road_randomization_params

    def generate_random_road_elements_list(self):
        params = self.road_randomization_params or {}
        num_elements_range = params.get('num_elements_range', (2, 5))
        straight_length_range = params.get('straight_length_range', (50.0, 200.0))
        curvature_range = params.get('curvature_range', (-1/500.0, 1/500.0))
        angle_range = params.get('angle_range', (10.0, 60.0))

        road_elements = []
        num_elements = random.randint(*num_elements_range)

        for _ in range(num_elements):
            element_type = random.choice(['straight', 'curved'])
            if element_type == 'straight':
                length = random.uniform(*straight_length_range)
                road_elements.append({"type": "straight", "length": length})
            else:
                curvature = random.uniform(*curvature_range)
                angle = random.uniform(*angle_range)
                road_elements.append({
                    "type": "curved",
                    "curvature": curvature,
                    "angle_in_degrees": angle
                })
        return road_elements

    def reset(self, **kwargs):
        # Generate a new random road
        self.unwrapped.road_elements_list = self.generate_random_road_elements_list()
        self.unwrapped.road = Road(epsilon_c=(1/10000), road_elements_list=self.unwrapped.road_elements_list)
        self.unwrapped.total_road_length = self.unwrapped.road.get_total_length()
        self.unwrapped.total_road_length_for_termination = max(self.unwrapped.total_road_length - 0.1, 0.9999 * self.unwrapped.total_road_length)
        # Call the original reset method
        return self.env.reset(**kwargs)

### Visualizing Random Roads:

In [ ]:
# Example 1: Standard Randomization
# road_randomization_params = {
#     'num_elements_range': (3, 6),          # Road will have 3 to 6 elements
#     'straight_length_range': (100.0, 150.0), # Length of straight segments between 100 and 150 meters
#     'curvature_range': (-1/300.0, 1/300.0), # Curvature between -1/300 and 1/300 for curves
#     'angle_range': (20.0, 45.0)            # Angle of the curves between 20 and 45 degrees
# }

# # Example 2: More Straight Roads
# road_randomization_params = {
#     'num_elements_range': (2, 4),            # Fewer elements, 2 to 4
#     'straight_length_range': (150.0, 300.0), # Longer straight segments between 150 and 300 meters
#     'curvature_range': (-1/1000.0, 1/1000.0), # Small curvature for gentler curves
#     'angle_range': (10.0, 30.0)              # Curves have lower angles between 10 and 30 degrees
# }

# # Example 3: More challenging roads with Tighter curves
# road_randomization_params = {
#     'num_elements_range': (4, 8),             # More road elements
#     'straight_length_range': (50.0, 100.0),   # Shorter straight segments
#     'curvature_range': (-1/100.0, 1/100.0),   # Tighter curves
#     'angle_range': (30.0, 70.0)               # More aggressive curves with wider angle range
# }

# Example 4: Completely Randomized
road_randomization_params = {
    'num_elements_range': (1, 10),             # Highly variable number of road elements
    'straight_length_range': (50.0, 300.0),    # Straight segments can be very short or long
    'curvature_range': (-1/50.0, 1/50.0),      # Very tight curves possible
    'angle_range': (10.0, 90.0)                # Wide range of angles for curves
}

random_env = create_env(road_elements_list = road_elements_list, ) # An initial road_elements_list is required but will be overwritten
random_env = DomainRandomizationWrapper(random_env, road_randomization_params=road_randomization_params)

- [ ] Define your own / Choose a different road randomization parameter

Reset the random env - this will create a new random road

In [ ]:
random_env.reset()

# Render the road
random_env.unwrapped.render_matplotlib_init_figure()
random_env.unwrapped.render_matplotlib_plot_road()
plt.show()

### Setting up Training/Evaluation Environments:

In [ ]:
### ------------- MAKE CHANGES TO THE ROAD RANDOMIZATION PARAMS HERE ------- ###
road_randomization_params = None    # Use None for default randomization params
# road_randomization_params = {
#     "num_elements_range": (0, 0),
#     "straight_length_range": (0.0, 0.0),
#     "curvature_range": (-0/100.0, 0/100.0),
#     "angle_range": (0.0, 0.0),
# }

### ------------------------------------------------------------------------ ###

# Create the training environment with domain randomization
train_env = create_env(road_elements_list=road_elements_list)  # Road elements will be randomized
train_env = DomainRandomizationWrapper(train_env, road_randomization_params = road_randomization_params)
train_env = CombinedRewardWrapper(train_env)

Difficult road for evaluation:

In [ ]:
# Define a fixed, difficult road for evaluation
road_elements_list_difficult = [
    {"type": "straight", "length": 50.0, 'v_max_kph': 60.0},
    {"type": "curved", "curvature": 1/300.0, "angle_in_degrees": 90.0, 'v_max_kph': 40.0},
    {"type": "straight", "length": 50.0, 'v_max_kph': 30.0, 'v_max_kph': 50.0},
    {"type": "curved", "curvature": -1/200.0, "angle_in_degrees": 120.0, 'v_max_kph': 40.0},
    {"type": "straight", "length": 50.0, 'v_max_kph': 60.0},
]

# Create the evaluation environment with the fixed difficult road
eval_env = create_env(road_elements_list_difficult)

# Render the road
eval_env.unwrapped.render_matplotlib_init_figure()
eval_env.unwrapped.render_matplotlib_plot_road()
plt.show()


### Model Definition and Training:

In [ ]:
from stable_baselines3 import PPO, SAC, DDPG

model_name = "PPO_dr"

TIMESTEPS_PER_EPOCH = 50000
EPOCHS = 20

logdir = "logs"
models_dir = f"models/{model_name}"
figs_dir_train = f"models/{model_name}/figs_train"
figs_dir_test = f"models/{model_name}/figs_test"

ensure_dirs([logdir, models_dir, figs_dir_train, figs_dir_test])

model = PPO("MultiInputPolicy", train_env, verbose = 1, tensorboard_log=logdir)

for i in range(1, EPOCHS+1):
    model.learn(
        total_timesteps=TIMESTEPS_PER_EPOCH,
        reset_num_timesteps=False,
        tb_log_name=f"{model_name}"
    )
    model.save(f"{models_dir}/{TIMESTEPS_PER_EPOCH * i}")
    # Evaluate on the most recent training road
    eval_model(train_env, model, figs_dir_train, TIMESTEPS_PER_EPOCH * i, plot_velocity=True)
    # Evaluate on evaluation road
    eval_model(eval_env, model, figs_dir_test, TIMESTEPS_PER_EPOCH * i, plot_velocity=True)

### Evaluating the RL Policy:

#### Load an RL model from the training loop:

In [ ]:
model_name = "PPO_dr"
model_idx = 950000

path_for_saving_figures = f"models/{model_name}/eval"
ensure_dir(path_for_saving_figures)

#### Put the RL model into a policy class with a 'standardized' interface

In [ ]:
from policies.rl_policy import RLPolicy

# Put the RL model into a policy class
rl_policy = RLPolicy(model)

#### Perform One simulation of the policy, plot the time series results:

In [ ]:
# Import the function for simulating the autonomous driving environment
from evaluation.evaluation_for_autonomous_driving import simulate_policy
from evaluation.evaluation_for_autonomous_driving import plot_results_from_time_series_dict

# Specify the length of the simulation in time steps
# > If termination or truncation is flagged by the "step" function,
#   Then, the simulation ends.
N_sim = 5000

# Specify the seed for when the simulate function resets the random number generator
sim_seed = 1;

# Call the function for simulating a given RL model
sim_time_series_dict = simulate_policy(eval_env, N_sim, rl_policy, seed=sim_seed, verbose=1)

# Call the plotting function
file_name_suffix = "example"
plot_details_list = plot_results_from_time_series_dict(eval_env, sim_time_series_dict, path_for_saving_figures, file_name_suffix, should_plot_reward=True)

#### Animate the time series results:

In [ ]:
def animate_from_sim_time_series_dict(sim_time_series_dict, Ts, path_for_saving_figures):
    # Extract the necessary trajectory information from the "sim_time_series_dict"
    px_traj    = sim_time_series_dict["px"]
    py_traj    = sim_time_series_dict["py"]
    theta_traj = sim_time_series_dict["theta"]
    delta_traj = sim_time_series_dict["delta"]
    # Call the environments function to create the simulation
    ani = eval_env.unwrapped.render_matplotlib_animation_of_trajectory(px_traj, py_traj, theta_traj, delta_traj, Ts, traj_increment=3)
    # Save the animation
    ani.save(f"{path_for_saving_figures}/trajectory_animation.gif")
    print(f'Saved animation at {path_for_saving_figures}/trajectory_animation.gif')
    # Return the animation object
    return ani

In [ ]:
# Call the animation functoin
ani = animate_from_sim_time_series_dict(sim_time_series_dict, numerical_integration_parameters["Ts"], path_for_saving_figures)
# Display the animation
from IPython.display import HTML
HTML(ani.to_jshtml())

### Performance Metrics:

#### Define a performance Metrics Function per simulation time series:

In [ ]:
def compute_performance_metrics_from_time_series(sim_time_series_dict):
    # Compute the statistics of the distance to the line
    # > Note that this is signed value, hence need to take absolute value of the data.
    abs_dist_to_line_time_series = np.abs(sim_time_series_dict["distance_to_closest_point"])
    avg_dist  = np.nanmean(abs_dist_to_line_time_series)
    std_dist  = np.nanstd(abs_dist_to_line_time_series)
    max_dist  = np.nanmax(abs_dist_to_line_time_series)

    # Compute the statistics of the speed in the forward direction (i.e., the body-frame x-axis direction)
    speed_time_series = np.abs(sim_time_series_dict["vx"])
    avg_speed = np.nanmean(speed_time_series)
    std_speed = np.nanstd(speed_time_series)
    max_speed = np.nanmax(speed_time_series)
    min_speed = np.nanmin(speed_time_series)

    # Return the results
    return {
        "avg_dist"   :  avg_dist,
        "std_dist"   :  std_dist,
        "max_dist"   :  max_dist,
        "avg_speed"  :  avg_speed * 3.6,
        "std_speed"  :  std_speed * 3.6,
        "max_speed"  :  max_speed * 3.6,
        "min_speed"  :  min_speed * 3.6,
    }

#### Run the Performance Metric Function for one simulation:

In [ ]:
# Simulate with the same seed as above to check that it gets the results consistent
sim_seed = 1;
sim_time_series_dict = simulate_policy(eval_env, N_sim, rl_policy, seed=sim_seed, verbose=1)

# Call the function for computing the performance metrics
pm_dict = compute_performance_metrics_from_time_series(sim_time_series_dict)

# Display the performance metric values
print("Performance Metric dictionary:")
print(pm_dict)

**NOTE:**
> Please refer to the notebook `ad_gym_rl_reward_shaping_v2025_08_25.ipynb` for more details on evaluation and performance metrics.